# 异常对象

except分句列出⼀个通⽤的异常父类，就可捕捉整个类别中的各种异常，任何特定的⼦类都可以匹配

In [1]:
class General(Exception): ...
class Specific1(General): ...
class Specific2(General): ...

In [9]:
for cls in (General, Specific1, Specific2):
    try:
        raise cls
    except General as X:
        import sys
        print(f'caught {sys.exc_info()}|{X.__class__}')

caught (<class '__main__.General'>, General(), <traceback object at 0x1094abfc0>)|<class '__main__.General'>
caught (<class '__main__.Specific1'>, Specific1(), <traceback object at 0x1094abfc0>)|<class '__main__.Specific1'>
caught (<class '__main__.Specific2'>, Specific2(), <traceback object at 0x1094abfc0>)|<class '__main__.Specific2'>


In [3]:
Exception.__base__

BaseException

In [11]:
I = IndexError('hello', 'no')

In [13]:
I

IndexError('hello', 'no')

In [14]:
I.args

('hello', 'no')

In [15]:
Z = ZeroDivisionError('Bad')
Z.args

('Bad',)

参数传递

In [16]:
class FormatError(Exception): 
    def __init__(self, line, file):
        self.line = line
        self.file = file

In [17]:
def parser():
    raise FormatError(42, 'spam.txt')

In [21]:
try:
    parser()
except FormatError as X:
    print(f'Error at line {X.line} from <{X.file}>')
    print(f'{X.args[0]}, {X.args[1]}')

Error at line 42 from <spam.txt>
42, spam.txt


提供异常⽅法

In [23]:
class FormatError(Exception):
    logfile = 'error.log'
    def __init__(self, line, file):
        self.line = line
        self.file = file
    def log(self):
        with open(self.logfile, 'a') as f:
            f.write(f'Error at line {self.line} from <{self.file}>\n')

In [26]:
try:
    raise FormatError(42, 'spam.txt')
except FormatError as X:
    X.log()

In [27]:
with open(FormatError.logfile) as f:
    print(f.read())

Error at line 42 from <spam.txt>
Error at line 42 from <spam.txt>
Error at line 42 from <spam.txt>



In [28]:
dir(Exception)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'add_note',
 'args',
 'with_traceback']

# 控制流

In [29]:
def action1():
    try:
        raise TypeError
    except TypeError:
        print('inner try')
        
try:
    action1()
except TypeError:
    print('outer try')

inner try


In [30]:
try:
    try:
        raise IndexError
    finally:
        print('inner try')
finally:
    print('outer try')

inner try
outer try


IndexError: 

In [32]:
def raise1(): raise IndexError
def raise2(): raise SyntaxError
def noraise(): return

for func in (raise1, noraise, raise2):
    print(f'<{func.__name__}>')
    try:
        try:
            func()
        except IndexError:
            print('caught IndexError')
    finally:
        print('finally run')
    print('...')

<raise1>
caught IndexError
finally run
...
<noraise>
finally run
...
<raise2>
finally run


SyntaxError: None (<string>)

跳出多重嵌套循环：“go to”

Python 的break 语句只能跳出⼀个最近的外围循环，但我们总可利用异常跳出更多层的循环：

In [34]:
class ExitLoop(Exception): pass

try:
    while True:
        for i in range(10):
            if i > 3:
                raise ExitLoop # 跳出内层循环
            print(f'loop3 : {i}')
        print('loop2')
    print('loop1')
except ExitLoop:
    print('countinuing')

loop3 : 0
loop3 : 1
loop3 : 2
loop3 : 3
countinuing


In [35]:
i

4

## 异常不全都是错误
在Python中，所有错误都是异常，但并非所有异常都是错误
> `EOFError`作为文件结束的信号

In [1]:
while True:
    try:
        line = input('> ')
    except EOFError:
        break
    else:
        print(f'you entered {line}')
        if line == 'quit':
            break

you entered quit


自定义信号

In [6]:
class Found(Exception): pass

def searcher():
    if True:
        raise Found
    else:
        return
    
try:
    searcher()
except Found:
    print('found')
else:
    print('not found')

found


In [4]:
class Failure(Exception): pass

def searcher():
    if True:
        return 3
    else:
        raise Failure
    
try:
    item = searcher()
except Failure:
    print('failure')
else:
    print(f'found {item}')

found 3


In [9]:
try:
    raise ZeroDivisionError from ArithmeticError
except FloatingPointError as X:
    print(f'{X.__class__.__name__} from {X.__cause__.__class__.__name__}')

ZeroDivisionError: 

In [21]:
import traceback as tb
import sys
try:
	raise OSError
except (ValueError, OSError) as e:
    print('调用方法method1处理')		
    print(sys.exc_info())
    print(e.__class__)
    tb.print_exc()		#堆栈跟踪

调用方法method1处理
(<class 'OSError'>, OSError(), <traceback object at 0x10713bd00>)
<class 'OSError'>


Traceback (most recent call last):
  File "/var/folders/l6/lzt7hk8j2873k3xp1_cpjcbh0000gn/T/ipykernel_18178/22511347.py", line 4, in <module>
    raise OSError
OSError


Python 也提供⼀个 sys.exit(statuscode) 内置调⽤来提前终⽌脚本。这实际上是引发内置的 SystemExit 异常来 终⽌程序，使try/finally 可以在离开前执⾏，并且让特殊类型的程序能拦截该事件

In [22]:
try:
    sys.exit(40)
except:
    print('caught')
print('continuing')

caught
continuing


In [23]:
try:
    sys.exit(40)
except Exception:
    print('caught')
print('continuing')

SystemExit: 40

/Users/kearney/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [24]:
try:
    sys.exit(40)
except SystemExit:
    print('caught')
print('continuing')

caught
continuing
